#Problem 1 - Creating 1D-Convolutional Layer

In [ ]:
class SimpleConv1d():

    def forward(self, x, w, b):
      a = []
      for i in range(len(w) - 1):
          a.append((x[i:i+len(w)] @ w) + b[0])
      return np.array(a)

    def backward(self, x, w, da):
        db = np.sum(da)
        dw = []
        for i in range(len(w)):
            dw.append(da @ x[i:i+len(da)])
        dw = np.array(dw)
        dx = []
        new_w = np.insert(w[::-1], 0, 0)
        new_w = np.append(new_w, 0)
        for i in range(len(new_w)-1):
            dx.append(new_w[i:i+len(da)] @ da)
        dx = np.array(dx[::-1])
        return db, dw, dx

#Problem 2 - Output Size calculator

In [ ]:
def output_size_calculator(n_in, P, F, S):
  n_out = int((n_in + 2*P - F) / S + 1)
  return n_out

#Problem 3 - Experimentation of the 1D CL

In [ ]:
import numpy as np

x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])

In [ ]:
a = np.array([35, 50])

In [ ]:
delta_a = np.array([10, 20])

In [ ]:
delta_b = np.array([30])
delta_w = np.array([50, 80, 110])
delta_x = np.array([30, 110, 170, 140])

In [ ]:
import numpy as np
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])
da = np.array([10, 20])


In [ ]:
SC1D = SimpleConv1d()
db, dw, dx = SC1D.backward(x, w, da)
print(SC1D.forward(x, w, b))
print(db)
print(dw)
print(dx)

[35 50]
30
[ 50  80 110]
[ 30 110 170 140]


#Problem 4 - Creating a one dimensional convo. layerclass that doesn't limitthe number of channels

In [ ]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape (2, 4), (Number of input channels, number of features).
w = np.ones((3, 2, 3))
b = np.array([1, 2, 3])

In [ ]:
a = np.array([[16, 22], [17, 23], [18, 24]])

In [ ]:
class GetMiniBatch:

    def __init__(self,x,y, batch_size = 20, seed=0):
      self.batch_size = batch_size
      np.random.seed(seed)
      shuffle_index = np.random.permutation(np.arange(x.shape[0]))
      self._x = x[shuffle_index]
      self._y = y[shuffle_index]
      self._stop = np.cell(x.shape[0]/self.batch_size).astype(np.int)

    def __len__(self):
        return self._stop
    
    def __getitem__(self, item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._x[p0:p1], self.y[p0:p1]
    
    def __iter__(self):
        self._counter = 0
        return self
    
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.x[p0:p1], self._y[p0:p1]

In [ ]:
class SimpleInitializer:

    def __init__(self, sigma):
        self.sigma =sigma
    
    def W(self, *shape):
        W = self.sigma * np.random.randn(*shape)
        return W
    
    def B(self, *shape):
        B = self.sigma * np.random.randn(*shape)
        return B

In [ ]:
class SGD:

    def __init__(self, lr):
        self.lr = lr
    
    def update(self, layer):
        layer.W -= self.lr * layer.dW / len(layer.Z)
        layer.B -= self.lr * layer.dB / len(layer.Z)
        return layer

In [ ]:
class AdaGrad:

    def __init__(self, lr):
        self.lr = lr
        self.HW = 1
        self.HB = 1

    def update(self,layer):
        self.HW += layer.dW**2
        self.HB += layer.dB**2
        layer.W -= self.lr *np.sqrt(1/self.HW) * layer.dW
        layer.B -= self.lr *np.sqrt(1/self.HB) * layer.dB

In [ ]:
def output_size_calculator(n_in, F, P=0, S=1):
    n_out = int((n_in + 2*P - F) / S + 1)
    return n_out

#Problem 5 - Implementing Padding

In [ ]:
class Conv1d:

    def __init__(self, b_size, initializer, optimizer, n_in_channels=1, n_out_channels=1, pa=0):
        self.b_size = b_size
        self.optimizer = optimizer
        self.pa = pa
        self.W = initializer.W(n_out_channels, n_in_channels, b_size)
        self.B = initializer.B(n_out_channels)
        self.n_in_channels = n_in_channels
        self.n_out_channels = n_out_channels
        self.n_out = None
    
    def forward(self, X):
        self.n_in = X.shape[-1]
        self.n_out = output_size_calculator(self.n_in, self.b_size, self.pa)
        X = X.reshape(self.n_in_channels, self.n_in)
        self.X = np.pad(X, ((0,0), ((self.b_size-1), 0)))
        self.X1 = np.zeros((self.n_in_channels, self.b_size, self.n_in+(self.b_size-1)))
        for i in range(self.b_size):
            self.X1[:, i] = np.roll(self.X, -1, axis=-1)
        A = np.sum(self.X1[:, :, self.b_size-1-self.pa:self.n_in+self.pa]*self.W[:, :, :, np.newaxis], axis=(1, 2)) + self.B.reshape(-1,1)
        return A
    
    def backward(self, dA):
        self.dW = np.sum(np.dot(dA, self.X1[:, :, self.b_size-1-self.pa:self.n_in+self.pa, np.newaxis]), axis=-1)
        self.dB = np.sum(dA, axis=1)
        self.dA = np.pad(dA, ((0,0), (0, (self.b_size-1))))
        self.dA1 = np.zeros((self.n_out_channels, self.b_size, self.dA.shape[-1]))
        for i in range(self.b_size):
          self.dA1[:, i] = np.roll(self.dA, i, axis=-1)
        dX = np.sum(self.W@self.dA1, axis=0)
        self.optimizer.update(self)
        print("dW:", self.dW)
        print("dB:", self.dB)
        print("dX:", dX)
        return dX

#Problem 6 - Response to Mini batch

In [ ]:
import numpy as np
test = Conv1d(b_size=3, initializer=SimpleInitializer(0.01), optimizer=AdaGrad(0.01), n_in_channels=2, n_out_channels=3, pa=0)

In [ ]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]])
test.W = np.ones((3, 2, 3), dtype=float)
test.B = np.array([1, 2, 3], dtype=float)

In [ ]:
testing = test.forward(x)
testing

array([[16., 22.],
       [17., 23.],
       [18., 24.]])

In [ ]:
test.backward(testing)

dW: [[[ 98.  98.  98.]
  [136. 136. 136.]]

 [[103. 103. 103.]
  [143. 143. 143.]]

 [[108. 108. 108.]
  [150. 150. 150.]]]
dB: [38. 40. 42.]
dX: [[ 51. 120. 120.  69.]
 [ 51. 120. 120.  69.]]


array([[ 51., 120., 120.,  69.],
       [ 51., 120., 120.,  69.]])

#Problem 7 - Arbitrary Number of Strides

In [ ]:
class Conv1d_Arbitrary_strides:

    def __init__(self, b_size, initializer, optimizer, n_in_channels=1, n_out_channels=1, pa=0, stride=1):
        self.b_size = b_size
        self.optimizer = optimizer
        self.pa = pa
        self.stride = stride
        self.W = initializer.W(n_out_channels, n_in_channels, b_size)
        self.B = initializer.B(n_out_channels)
        self.n_in_channels = n_in_channels
        self.n_out_channels = n_out_channels
        self.n_out = None
    
    def forward(self, X):
        self.n_samples = X.shape[0]
        self.n_in = X.shape[-1]
        self.n_out = output_size_calculator(self.n_in, self.b_size, self.pa, self.stride)
        X = X.reshape(self.n_samples, self.n_in_channels, self.n_in)
        self.X = np.pad(X, ((0,0), (0,0), ((self.b_size-1), 0)))
        self.X1 = np.zeros((self.n_samples, self.n_in_channels, self.b_size, self.n_in+(self.b_size-1)))
        for i in range(self.b_size):
            self.X1[:, :, i] = np.roll(self.X, -1, axis=-1)
        A = np.sum(self.X1[:, np.newaxis, :, :, self.b_size-1-self.pa:self.n_in+self.pa:self.stride]*self.W[:, :, :, np.newaxis], axis=(2, 3)) + self.B.reshape(-1,1)
        return A
    
    def backward(self, dA):
        self.dW = np.sum(dA[:, :, np.newaxis, np.newaxis]*self.X1[:, np.newaxis, :, :, self.b_size-1-self.pa:self.n_in+self.pa:self.stride], axis=(0, -1))
        self.dB = np.sum(dA, axis=(0, -1))
        self.dA = np.pad(dA, ((0,0), (0,0), (0, (self.b_size-1))))
        self.dA1 = np.zeros((self.n_samples, self.n_out_channels, self.b_size, self.dA.shape[-1]))
        for i in range(self.b_size):
          self.dA1[:, :, i] = np.roll(self.dA, i, axis=-1)
        dX = np.sum(self.W[:, :, :, np.newaxis]*self.dA1[:, :, np.newaxis], axis=(1,3))
        self.optimizer.update(self)
        return dX

#Problem 8 - Learning and Estimation

In [ ]:
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

x_train = x_train.astype(np.float)
x_test = x_test.astype(np.float)
x_train /= 255
x_test /= 255

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
print(y_train.shape) 
print(y_train_one_hot.shape)
print(y_train_one_hot.dtype)

11490434/11490434 [==============================] - 0s 0us/step


<ipython-input-22-c9d2fc207084>:18: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x_train = x_train.astype(np.float)


(60000,)
(60000, 10)
float64


<ipython-input-22-c9d2fc207084>:19: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x_test = x_test.astype(np.float)


In [ ]:
x_train_, x_val, y_train_, y_val = train_test_split(x_train, y_train_one_hot, test_size=0.2)
print("x_train.shape =", x_train_.shape)
print("x_val.shape =", x_val.shape)
print("y_train.shape =", y_train_.shape)
print("y_val.shape =", y_val.shape)

x_train.shape = (48000, 784)
x_val.shape = (12000, 784)
y_train.shape = (48000, 10)
y_val.shape = (12000, 10)


In [ ]:
class GetMiniBatch:
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1] 
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [ ]:
get_mini_batch = GetMiniBatch(x_train, y_train, batch_size=20)
for mini_x_train, mini_y_train in get_mini_batch:
    pass

<ipython-input-24-01a04e45f93e>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)


In [ ]:
test2 = Conv1d_Arbitrary_strides(b_size=3, initializer=SimpleInitializer(0.01), optimizer=SGD(0.01), n_in_channels=1, n_out_channels=1, pa=1)
testing2 = test2.forward(mini_x_train)
print(testing2.shape)

(20, 1, 784)


In [ ]:
import math

class Sigmoid:

    def forward(self, A):
        self.A = A
        return self.sigmoid(A)
    
    def backward(self, dZ):
        _sig = self.sigmoid(self.A)
        return dZ * (1 - _sig)*_sig
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

class Tanh:

    def forward(self, A):
        self.A = A
        return np.tanh(A)
    
    def backward(self, dZ):
        return dZ * (1 - np.tanh(self.A)**2)

class Softmax:
    
    def forward(self, X):
        self.Z = np.exp(X) / np.sum(np.exp(X), axis=1).reshape(-1,1)
        return self.Z
    
    def backward(self, Y):
        self.loss = self.loss_func(Y)
        return self.Z - Y
    
    def loss_func(self, Y, Z=None):
        if Z is None:
            Z = self.Z
        return (-1)*np.average(np.sum(Y*np.log(Z), axis=1))

class ReLU:

    def forward(self, A):
        self.A = A
        return np.clip(A, 0, None)

    def backward(self, dZ):
        return dZ * np.clip(np.sign(self.A), 0, None)

class FC:
    
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.W = initializer.W(n_nodes1, n_nodes2)
        self.B = initializer.B(n_nodes2)
        
    def forward(self, X):
        self.X = X
        A = X@self.W + self.B
        return A
    
    def backward(self, dA):
        dZ = dA@self.W.T
        self.dB = np.sum(dA, axis=0)
        self.dW = self.X.T@dA
        self.optimizer.update(self)
        return dZ        

class XavierInitializer:
    
    def W(self, n_nodes1, n_nodes2):
        self.sigma = math.sqrt(1 / n_nodes1)
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W
    
    def B(self, n_nodes2):
        B = self.sigma * np.random.randn(n_nodes2)
        return B

class HeInitializer:

    def W(self, n_nodes1, n_nodes2):
        self.sigma = math.sqrt(2 / n_nodes1)
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W
    
    def B(self, n_nodes2):
        B = self.sigma * np.random.randn(n_nodes2)
        return B

class SGD:

    def __init__(self, lr):
        self.lr = lr
    
    def update(self, layer):
        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB
        return

class AdaGrad:

    def __init__(self, lr):
        self.lr = lr
        self.HW = 1
        self.HB = 1

    def update(self,layer):
        self.HW += layer.dW**2
        self.HB += layer.dB**2
        layer.W -= self.lr *np.sqrt(1/self.HW) * layer.dW
        layer.B -= self.lr *np.sqrt(1/self.HB) * layer.dB

Scratch Convolutional Neural Net. Classifier

In [ ]:
import math
class ScratchCNNClassifier:
    
    def __init__(self, num_epoch=10, lr=0.01, batch_size=20, n_features=784, n_nodes1=400, n_nodes2=200, n_output=10, verbose=True, Activater=Tanh, Optimizer=AdaGrad):
        self.num_epoch = num_epoch
        self.lr = lr
        self.verbose = verbose  
        self.batch_size = batch_size 
        self.n_features = n_features 
        self.n_nodes2 = n_nodes2 
        self.n_output = n_output 
        self.Activater = Activater
        if Activater == Sigmoid or Activater == Tanh:
            self.Initializer = XavierInitializer
        elif Activater == ReLU:
            self.Initializer = HeInitializer
        self.Optimizer = Optimizer
    
    def fit(self, X, y, X_val=None, y_val=None):
        self.val_enable = False
        if X_val is not None:
            self.val_enable = True
        self.Conv1d_Arbitrary_Strides = Conv1d_Arbitrary_strides(b_size=7, initializer=SimpleInitializer(0.01), optimizer=self.Optimizer(self.lr), n_in_channels=1, n_out_channels=1, pa=3, stride=2)
        self.Conv1d_Arbitrary_Strides.n_out = output_size_calculator(X.shape[-1], self.Conv1d_Arbitrary_Strides.b_size, self.Conv1d_Arbitrary_Strides.pa, self.Conv1d_Arbitrary_Strides.stride)
        self.activation1 = self.Activater()
        self.FC2 = FC(1*self.Conv1d_Arbitrary_Strides.n_out, self.n_nodes2, self.Initializer(), self.Optimizer(self.lr))
        self.activation2 = self.Activater()
        self.FC3 = FC(self.n_nodes2, self.n_output, self.Initializer(), self.Optimizer(self.lr))
        self.activation3 = Softmax()
        
        self.loss = []
        self.loss_epoch = [self.activation3.loss_func(y, self.forward_propagation(X))]
        
        for _ in range(self.num_epoch):
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.batch_size)
            self.iter = len(get_mini_batch)
            for mini_X, mini_y in get_mini_batch:
                self.forward_propagation(mini_X)
                self.back_propagation(mini_X, mini_y)
                self.loss.append(self.activation3.loss)
            self.loss_epoch.append(self.activation3.loss_func(y, self.forward_propagation(X)))
        
    def predict(self, X):
        return np.argmax(self.forward_propagation(X), axis=1)
    
    def forward_propagation(self, X):
        A1 = self.Conv1d_Arbitrary_Strides.forward(X)
        A1 = A1.reshape(A1.shape[0], A1.shape[-1])
        Z1 = self.activation1.forward(A1)
        A2 = self.FC2.forward(Z1)
        Z2 = self.activation2.forward(A2)
        A3 = self.FC3.forward(Z2)
        Z3 = self.activation3.forward(A3)
        return Z3
        
    def back_propagation(self, X, y_true):
        dA3 = self.activation3.backward(y_true) 
        dZ2 = self.FC3.backward(dA3)
        dA2 = self.activation2.backward(dZ2)
        dZ1 = self.FC2.backward(dA2)
        dA1 = self.activation1.backward(dZ1)
        dA1 = dA1[:, np.newaxis]
        dZ0 = self.Conv1d_Arbitrary_strides.backward(dA1) 

In [ ]:
test3 = ScratchCNNClassifier(num_epoch=20, lr=0.01, batch_size=20, n_features=784, n_nodes1=400, n_nodes2=400, n_output=10, verbose=True, Activater=Tanh, Optimizer=SGD)
test3.fit(x_train_, y_train_)

<ipython-input-24-01a04e45f93e>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)


In [ ]:
y_pred = test3.predict(x_test)
accuracy_score(y_test, y_pred)

0.9765